## Setup

1. Complete the previous challenges.

### Create a Custom Azure Function

1. Create a Python Azure Function using the following code:

---
```
import logging
import os
import sys
import json
from string import punctuation
from collections import Counter
import azure.functions as func


def main(req: func.HttpRequest) -> func.HttpResponse:
    logging.info('Python HTTP trigger function processed a request.')

    # The result will be a "values" bag
    result = {
        "values": []
    }
    statuscode = 200

    # We're going to exclude words from this list in the word counts
    stopwords = ['', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
                "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 
                'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 
                'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 
                'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 
                'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was',
                'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 
                'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 
                'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
                'about', 'against', 'between', 'into', 'through', 'during', 'before', 
                'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 
                'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 
                'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 
                'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
                'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 
                'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll',
                'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 
                'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
                'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', 
                "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', 
                "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

    try:
        values = req.get_json().get('values')
        logging.info(values)

        for rec in values:
            # Construct the basic JSON response for this record
            val = {
                    "recordId": rec['recordId'],
                    "data": {
                        "text":None
                    },
                    "errors": None,
                    "warnings": None
                }
            try:
                # get the text to be processed from the input record
                txt = rec['data']['text']
                # remove numeric digits
                txt = ''.join(c for c in txt if not c.isdigit())
                # remove punctuation and make lower case
                txt = ''.join(c for c in txt if c not in punctuation).lower()
                # remove stopwords
                txt = ' '.join(w for w in txt.split() if w not in stopwords)
                # Count the words and get the most common 10
                wordcount = Counter(txt.split()).most_common(10)
                words = [w[0] for w in wordcount]
                # Add the top 10 words to the output for this text record
                val["data"]["text"] = words
            except:
                # An error occured for this text record, so add lists of errors and warning
                val["errors"] =[{"message": "An error occurred processing the text."}]
                val["warnings"] = [{"message": "One or more inputs failed to process."}]
            finally:
                # Add the value for this record to the response
                result["values"].append(val)
    except Exception as ex:
        statuscode = 500
        # A global error occurred, so return an error response
        val = {
                "recordId": None,
                "data": {
                    "text":None
                },
                "errors": [{"message": ex.args}],
                "warnings": [{"message": "The request failed to process."}]
            }
        result["values"].append(val)
    finally:
        # Return the response
        return func.HttpResponse(body=json.dumps(result), mimetype="application/json", status_code=statuscode)


```
---

2. Deploy the function to Azure. Then, in the **Test** pane, replace the default request body with the following JSON, and then click **Run**. Your function should return an array of text values representing the most common words in the input text:

```
{
    "values": [
        {
            "recordId": "a1",
            "data":
            {
               "text":  "Tiger, tiger burning bright in the darkness of the night.",
               "language": "en"
            }
        },
        {
            "recordId": "a2",
            "data":
            {
               "text":  "The rain in spain stays mainly in the plains! That's where you'll find the rain!",
               "language": "en"
            }
        }
   ]
}
```
3. Above your function code, click **</> Get function URL**, and copy the default (\[Function key)\] URL to the clipboard. Then paste the copied function key in the code below to replace the ***function_url*** constant. NThen run the cell below and update the skillset to include a new custom skill that passes the text from each document to your custom function and extracts the top ten words.

In [1]:
# Azure Storage account settings:
azure_storage_account = 'YOUR_STORAGE_ACCOUNT'
blob_connection_string = "YOUR_BLOB_STORE_CONNECTION_STRING"
blob_container = "YOUR_BLOB_CONTAINER"

# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_SEARCH_KEY"

# Cognitive Services Key
cog_svc_key = "YOUR_COGNITIVE_SERVICE_KEY"

# Function URL
function_url = 'YOUR_FUNCTION_URL'

## Function to wrap HTTP requests

In [2]:
def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2019-05-06-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


## Update the Index

In [5]:
import json

# Define the request body
body = {
  "fields": [
    {
      # Unique ID for each document
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
     # The file path of the document
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The file name of the document
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The size of the document
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The last-modified date of the document
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The text content in the document
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The sentiment score
      "name": "sentiment",
      "type": "Edm.Double",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": True
    },
    {
      # Key phrases that might help identify the subject of the text
      "name": "key_phrases",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "filterable": False,
      "facetable": False
    },
    {
      # Geographical locations mentioned in the text
      "name": "locations",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # People mentioned in the text
      "name": "people",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "links",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # URLs mentioned in the text
      "name": "entities",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },    {
      # The image content in the document
      "name": "image_descriptions",
      "type": "Collection(Edm.String)",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The text contained in images in the document
      "name": "image_text",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The combined source text plus the text extracted from images
      "name": "merged_text",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": False,
      "facetable": False
    },
    {
      # The most common words in the text
      "name": "top_words",
      "type": "Collection(Edm.String)",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    }
  ]
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/website-docs-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

204


## Update the Skillset

In [7]:
# Define the request body
body = {
  "description": "Extract and enhance text",
  "skills":
  [
      # Detect the language
      {
          "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
          "inputs": [
            {
              "name": "text", "source": "/document/content"
            }
          ],
            "outputs": [
                {
                  "name": "languageCode",
                  "targetName": "languageCode"
                }
            ]
        },
      
      # Extract text from images (including those embedded in documents)
        {
            "description": "Extract descriptions from images.",
            "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "visualFeatures": ["Description"],
            "details": ["Landmarks"],
            "inputs": [
              {
                "name": "image",
                "source": "/document/normalized_images/*"
              }
            ],
            "outputs": [
              {
                "name": "description",
                "targetName": "imageDescription"
              }
            ]
        },
      # Extract text from images (including those embedded in documents)
        {
            "description": "Extract text from images.",
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
              {
                "name": "image",
                "source": "/document/normalized_images/*"
              }
            ],
            "outputs": [
              {
                "name": "text"
              }
            ]
        },
       # Merge the extracted image text back into the content at the appropriate locations
        {
          "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
          "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field.",
          "context": "/document",
          "insertPreTag": "[",
          "insertPostTag": "]",
          "inputs": [
            {
              "name":"text", "source": "/document/content"
            },
            {
              "name": "itemsToInsert", "source": "/document/normalized_images/*/text"
            },
            {
              "name":"offsets", "source": "/document/normalized_images/*/contentOffset" 
            }
          ],
          "outputs": [
            {
              "name": "mergedText"
            }
          ]
        },
         # Detect the sentiment
        {
          "description": "Detect sentiment.",
          "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",
        "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/mergedText"
            }
          ],
            "outputs": [
                {
                  "name": "score",
                  "targetName": "sentimentScore"
                }
            ]
        },
       # Extract entities
        {
          "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
          "categories": [ "Organization", "location", "person", "datetime", "url" ],
          "defaultLanguageCode": "en",
          "inputs": [
            {
              "name": "text",
              "source": "/document/mergedText"
            }
          ],
          "outputs": [
            {
              "name": "locations"
            },
            {
              "name": "persons"
            },
            {
              "name": "urls"
            },
            {
              "name": "entities"
            }
          ]
        },
      # Extract key phrases from the pages
      {
          "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
          "context": "/document",
          "defaultLanguageCode": "en",
          "maxKeyPhraseCount": 5,
          "inputs": [
            {
              "name": "text", "source": "/document/mergedText"
            }
          ],
          "outputs": [
            {
              "name": "keyPhrases"
            }
          ]
        }, 
       # Use a custom skill to find the most common words in each document
          {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "custom skill",
            "uri": function_url,
            "batchSize":1,
            "context": "/document",
            "inputs": [
              {
                "name": "text",
                "source": "/document/mergedText"
              },
            {
              "name":"language",
              "source": "/document/languageCode"
            }  
            ],
            "outputs": [
              {
                "name": "text",
                "targetName": "topWords"
              }
            ]
          }
      ],
      "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "margies-cog-svc",
        "key": cog_svc_key
      }
    }  

try:
    result = azsearch_rest(request_type="PUT", function_name="/skillsets/website-skillset", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


## Update the Indexer

In [9]:
import json

body = {
  "name":"website-docs-indexer", 
  "dataSourceName" : "website-docs",
  "skillsetName" : "website-skillset",
  "targetIndexName" : "website-docs-index",
  "fieldMappings" : [
      # Map the soure content fields to index fields
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : 
            { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "url"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "file_name"
        },
        {
          "sourceFieldName" : "content",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_size",
          "targetFieldName" : "size"
        },
        {
          "sourceFieldName" : "metadata_storage_last_modified",
          "targetFieldName" : "last_modified"
        }
   ],
  "outputFieldMappings" : 
  [
      # Map the output from cognitive skills to index fields
        {
            "sourceFieldName": "/document/normalized_images/*/imageDescription", 
            "targetFieldName": "image_descriptions"
        },
        {
            "sourceFieldName": "/document/normalized_images/*/text",
            "targetFieldName": "image_text"
        },
        {
            "sourceFieldName": "/document/mergedText", 
            "targetFieldName": "merged_text"
        },
        {
            "sourceFieldName": "/document/sentimentScore", 
            "targetFieldName": "sentiment"
        },
        {
          "sourceFieldName" : "/document/persons", 
          "targetFieldName" : "people"
        },
        {
          "sourceFieldName" : "/document/locations", 
          "targetFieldName" : "locations"
        },
        {
          "sourceFieldName" : "/document/urls", 
          "targetFieldName" : "links"
        },
        {
          "sourceFieldName" : "/document/entities", 
          "targetFieldName" : "entities"
        },
        {
          "sourceFieldName" : "/document/keyPhrases/*", 
          "targetFieldName" : "key_phrases"
        },
        {
          "sourceFieldName" : "/document/topWords", 
          "targetFieldName" : "top_words"
        }
  ],
  "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration": 
    {
        "dataToExtract": "contentAndMetadata",
        "imageAction": "generateNormalizedImages"
    }
  }
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexers/website-docs-indexer", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


In [10]:
import time

try:
    # Reset the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/reset", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
        
    # re-run the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/website-docs-indexer/run", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
    


except Exception as e:
    print('Error:')
    print(e)

204
202


In [11]:
import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/website-docs-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProg

## Create Wrapper Function for Query Requests

In [12]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


## Queries

### View Top Words

In [13]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, merged_text, top_words',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"])
        for word in value['top_words']:
            print(word)

except Exception as e:
    print('Error:')
    print(e)

Search: Las Vegas
Hits: 15

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Las%20Vegas%20Brochure.pdf
Las Vegas Brochure.pdf
vegas
las
city
hotel
margie’s
travel
known
populated
nevada
county

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201853.pdf
201853.pdf
hotel
vegas
strip
bedrooms
stay
rooms
would
massive
fountain
las

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201856.pdf
201856.pdf
hotel
las
vegas
may
icon
fountain
usa
reason
leave
property

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201845.pdf
201845.pdf
hotel
great
room
beds
volcano
las
vegas
view
also
comfy

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201844.pdf
201844.pdf
hotel
nice
rooms
top
enough
volcano
las
vegas
usa
really

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201859.pdf
201859.pdf
vegas
hotel
everyth

### Filter Based on Top Words

In [14]:
import urllib.parse, json

search_terms = input("Search: ")
filter_word = input("Keyword: ").lower()

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'{0}'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, file_name, merged_text, top_words',
    '$filter':"top_words/any(w: w eq '{0}')".format(filter_word),
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"])
        for word in value['top_words']:
            print(word)

except Exception as e:
    print('Error:')
    print(e)

Search: Las Vegas
Keyword: nice
Hits: 2

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201844.pdf
201844.pdf
hotel
nice
rooms
top
enough
volcano
las
vegas
usa
really

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201841.pdf
201841.pdf
hotel
rooms
location
volcano
also
awesome
nice
excellent
las
vegas
